<a href="https://colab.research.google.com/github/krithiyer/safe-elections/blob/main/cs294s_3A_ModelBased_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Uset BERT Model to determine similarity

Input both user query (question) and questions from the FAQ to the model to determine similarity

Input to model: [CLS] user query [SEP] question from FAQ [SEP}

Connect to google drive and load the saved model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/cs294s/

/content/drive/My Drive/cs294s


In [ ]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


install and import all needed packages

In [ ]:
pip install transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 18.3MB/s 
     |████████████████████████████████| 890kB 19.4MB/s 
     |████████████████████████████████| 3.0MB 24.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=12428820e54e4643d708ffe8b166d6b7773e3c0f470abcb076b7b11fa2a91e02
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

The model saved at './models_saved_qqp_oct9Final/' was fine tuned on QQP dataset for 4 epoch. 

In [ ]:


# The name of the folder containing the model files.
input_dir = './models_saved_qqp_oct9Final/'

# Load our fine-tuned model, and configure it to return the "hidden states", 
# from which we will be taking our text embeddings.
# num_labels = 2,  is this needed while loading saved models
model = BertForSequenceClassification.from_pretrained(
    input_dir,
    output_hidden_states = True, # Whether the model returns all hidden-states.
) 

# Load the tokenizer. we have not saved it.
#tokenizer = BertTokenizer.from_pretrained(output_dir)

model.cuda()

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

put the questions in right format and tokenize

In [ ]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 256  # if need be increase to 512

#user_query = "will i get covid if I vote"
#faq_question = "will i get covid if I vote"

def create_tokens (user_query, faq_question):
  input_ids = tokenizer.encode(
                        user_query, faq_question,                   # Sentence to encode.
                        add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN,       # Truncate all sentences. 
                        truncation = True                       
                   )  

  results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long", 
                              truncating="post", padding="post")
    
    # Remove the outer list.
  input_ids = results[0]

    # Create attention masks    
  attn_mask = [int(i>0) for i in input_ids]
    
    # Cast to tensors.
  input_ids = torch.tensor(input_ids)
  attn_mask = torch.tensor(attn_mask)

    # Add an extra dimension for the "batch" (even though there is only one 
    # input in this batch.)
  input_ids = input_ids.unsqueeze(0)
  attn_mask = attn_mask.unsqueeze(0)  

  return input_ids, attn_mask

In [ ]:
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]',
 'will',
 'i',
 'get',
 'co',
 '##vid',
 'if',
 'i',
 'vote',
 '[SEP]',
 'how',
 'will',
 'one',
 'get',
 'co',
 '##vid',
 '[SEP]']

In [ ]:
model.eval()

input_ids = input_ids.to(device)
attn_mask = attn_mask.to(device)
with torch.no_grad():
  logits, encoded_layers = model(input_ids = input_ids, token_type_ids = None, attention_mask = attn_mask)
probabilities = torch.nn.functional.softmax(logits, dim=1)
print (" logits : ", logits)
print (" probabilities : ", probabilities)

 logits :  tensor([[-1.2118,  1.2534]], device='cuda:0')
 probabilities :  tensor([[0.0783, 0.9217]], device='cuda:0')


In [ ]:
x = logits.cpu().numpy()

In [ ]:
x[0,1]

1.2533848

In [ ]:
p = torch.nn.functional.softmax(logits, dim=1)

In [ ]:
print(p)

tensor([[0.0783, 0.9217]], device='cuda:0', grad_fn=<SoftmaxBackward>)


In [ ]:
pwd

'/content/drive/My Drive/cs294s'

Initial curated QA consists of 1648 questions related to Covid Safety, Election Deadlines, etc.

Questions scraped from the Secretary of State web sites (election data)
and Covid safety data from different sites.

Some Covid related questions taken from  https://openreview.net/pdf?id=qd51R0JNLl   (not all questions used - some dont have answers)


In [ ]:
import pandas
election_faq_df = pandas.read_excel('./master_test1_v1_QA.xlsx')
election_faq_df.head()

,Question,Answer
0,"Due to COVID-19, can an election official requ...","No, an election official cannot require a vote..."
1,Can election officials require voters to wear ...,"No, you cannot require voters to wear face cov..."
2,May election officials require an assistant/in...,"No, just as you cannot require a voter to use ..."
3,If an election worker cannot identify a voter ...,The election judge has discretion to ask the v...
4,How should election officials sanitize equipme...,The SOS recommends that election officials san...


In [ ]:
proc_data_df = election_faq_df.dropna()
#proc_data_df = election_faq_df.head(5)

In [ ]:
proc_data_df.head()

,Question,Answer
0,"Due to COVID-19, can an election official requ...","No, an election official cannot require a vote..."
1,Can election officials require voters to wear ...,"No, you cannot require voters to wear face cov..."
2,May election officials require an assistant/in...,"No, just as you cannot require a voter to use ..."
3,If an election worker cannot identify a voter ...,The election judge has discretion to ask the v...
4,How should election officials sanitize equipme...,The SOS recommends that election officials san...


In [ ]:
from keras.preprocessing.sequence import pad_sequences

user_query = "May election officials require the use of a pencil with an eraser to mark on the electronic voting system?"

master_df = pandas.DataFrame(columns = ['User_Question', 'FAQ_Question', 'Logit_0', 'Logit_1', 'Probability_0', 'Probability_1'])
torch.no_grad()
for row in proc_data_df.itertuples():
  inpt_id, att_mask = create_tokens(user_query, row.Question)
  inpt_id = inpt_id.to(device)
  att_mask = att_mask.to(device)
  with torch.no_grad():
    logits, encoded_layers = model(input_ids = inpt_id, token_type_ids = None, attention_mask = att_mask)
  probabilities = torch.nn.functional.softmax(logits, dim=1)
  sclr_logits = logits.cpu().numpy()
  sclr_prob = probabilities.cpu().numpy()
  temp_df = pandas.DataFrame( [[ user_query, row.Question, sclr_logits[0][0], sclr_logits[0][1], sclr_prob[0][0], sclr_prob[0][1] ] ],
                             columns = ['User_Question', 'FAQ_Question', 'Logit_0', 'Logit_1', 'Probability_0', 'Probability_1'])

  master_df = master_df.append(temp_df)

master_df.to_excel ('./Query_faq_similarity.xlsx')

Streaming output truncated to the last 5000 lines.
0  May election officials require the use of a pe...  ...      0.000055

[1 rows x 6 columns]
                                       User_Question  ... Probability_1
0  May election officials require the use of a pe...  ...      0.000053

[1 rows x 6 columns]
                                       User_Question  ... Probability_1
0  May election officials require the use of a pe...  ...      0.000052

[1 rows x 6 columns]
                                       User_Question  ... Probability_1
0  May election officials require the use of a pe...  ...      0.000053

[1 rows x 6 columns]
                                       User_Question  ... Probability_1
0  May election officials require the use of a pe...  ...      0.000053

[1 rows x 6 columns]
                                       User_Question  ... Probability_1
0  May election officials require the use of a pe...  ...      0.000055

[1 rows x 6 columns]
                         

In [ ]:
master_df.head()

,User_Question,FAQ_Question,Logit_0,Logit_1,Probability_0,Probability_1
0,How can I get covid,"Due to COVID-19, can an election official requ...",-1.211812,1.253385,0.078334,0.921666
0,How can I get covid,Can election officials require voters to wear ...,-1.211812,1.253385,0.078334,0.921666
0,How can I get covid,May election officials require an assistant/in...,-1.211812,1.253385,0.078334,0.921666
0,How can I get covid,If an election worker cannot identify a voter ...,-1.211812,1.253385,0.078334,0.921666
0,How can I get covid,How should election officials sanitize equipme...,-1.211812,1.253385,0.078334,0.921666


In [ ]:
len(faq_q_embeddings)

1646